In [0]:
%sql
SELECT current_catalog(), current_schema()


In [0]:
%sql

USE CATALOG smart_claims_dev;
USE SCHEMA 01_bronze;

SELECT current_catalog(), current_schema()

In [0]:
%sql
SELECT *
FROM csv.`/Volumes/smart_claims_dev/00_landing/sql_server/claims.csv`
--drop the table if it exists for demonstration

In [0]:
%sql
DROP TABLE IF EXISTS smart_claims_dev.01_bronze.claims

##Create a STREAMING TABLE using Databricks Auto Loader


In [0]:
%sql

CREATE OR REFRESH STREAMING TABLE smart_claims_dev.01_bronze.claims
SCHEDULE EVERY 1 WEEK
AS
SELECT *
FROM read_files(
  '/Volumes/smart_claims_dev/00_landing/sql_server/claims.csv',
  format => 'csv',
  header => true
)


In [0]:
%sql
CREATE OR REFRESH STREAMING TABLE smart_claims_dev.01_bronze.claims
AS
SELECT *
FROM STREAM cloud_files(
  '/Volumes/smart_claims_dev/00_landing/sql_server/claims.csv',
  format => 'csv',
  header => true
)

In [0]:
%sql
-- This file defines a sample transformation.
-- Edit the sample below or add new transformations
-- using "+ Add" in the file browser.

CREATE OR REFRESH STREAMING LIVE TABLE smart_claims_dev.01_bronze.claims
COMMENT "Bronze — sinistros (claims) a partir de 00_landing/sql_server/claims.csv"
TBLPROPERTIES ("quality" = "bronze")
AS
SELECT
  *,
  _metadata.file_path               AS input_file,
  _metadata.file_modification_time  AS input_file_mtime,
  current_timestamp()               AS ingested_at
FROM cloud_files(
  "/Volumes/smart_claims_dev/00_landing/sql_server",
  "csv",
  map(
    "header", "true",
    "cloudFiles.inferColumnTypes", "true",
    "cloudFiles.schemaEvolutionMode", "addNewColumns",
    "pathGlobFilter", "claims.csv"
  )
);

In [0]:
%sql

SELECT * FROM smart_claims_dev.01_bronze.claims
LIMIT 10;